In [6]:
from keras import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [21]:
# datagen = ImageDataGenerator(rescale=1.0 / 255.0, validation_split=0.2)
datagen = ImageDataGenerator(rescale=1.0 / 255.0,
                             # rotation_range=20, shear_range=0.1,
                             # zoom_range=0.2, channel_shift_range=0.1, vertical_flip=True, width_shift_range=0.1,
                             # height_shift_range=0.1,
                             # validation_split=0.2
                             )
target_size = 240
train_it = datagen.flow_from_directory('ground_truth_small/train', class_mode='binary', batch_size=16, target_size=(target_size, target_size))
valid_it = datagen.flow_from_directory('ground_truth_small/test', class_mode='binary', batch_size=16, target_size=(target_size, target_size))


Found 3433 images belonging to 2 classes.
Found 389 images belonging to 2 classes.


In [22]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(target_size, target_size, 3)))
model.add(MaxPooling2D((10, 10)))
model.add(Conv2D(16, (3, 3), activation='relu'))
model.add(MaxPooling2D((7, 7)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4998909584788273168
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5738856448
locality {
  bus_id: 1
  links {
  }
}
incarnation: 14103149028267373437
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [23]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 236, 236, 32)      2432      
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 23, 23, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_13 (Conv2D)          (None, 21, 21, 16)        4624      
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 3, 3, 16)         0         
 g2D)                                                            
                                                                 
 flatten_6 (Flatten)         (None, 144)               0         
                                                                 
 dense_17 (Dense)            (None, 128)              

In [24]:
model.fit(train_it, validation_data=valid_it, epochs=5)

Epoch 1/5
215/215 [==============================] - 13s 58ms/step - loss: 0.2589 - accuracy: 0.8916 - precision_6: 0.8756 - recall_6: 0.9423 - val_loss: 0.2703 - val_accuracy: 0.9280 - val_precision_6: 0.9556 - val_recall_6: 0.8958
Epoch 2/5
215/215 [==============================] - 9s 41ms/step - loss: 0.0930 - accuracy: 0.9682 - precision_6: 0.9776 - recall_6: 0.9660 - val_loss: 0.4183 - val_accuracy: 0.7326 - val_precision_6: 0.6560 - val_recall_6: 0.9635
Epoch 3/5
215/215 [==============================] - 9s 41ms/step - loss: 0.0818 - accuracy: 0.9700 - precision_6: 0.9752 - recall_6: 0.9717 - val_loss: 0.4434 - val_accuracy: 0.7609 - val_precision_6: 0.6813 - val_recall_6: 0.9688
Epoch 4/5
215/215 [==============================] - 9s 42ms/step - loss: 0.0625 - accuracy: 0.9805 - precision_6: 0.9850 - recall_6: 0.9804 - val_loss: 0.4647 - val_accuracy: 0.7661 - val_precision_6: 0.6836 - val_recall_6: 0.9792
Epoch 5/5
215/215 [==============================] - 9s 44ms/step - los

In [ ]:
model.save('models/clasiffication_custom_augmentation_240_val_newdataset.h5')

In [ ]:
model.evaluate(valid_it)